In [1]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms, models
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [2]:
device = "cuda"
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss_sum += val_loss_fn(out, y).item()
            preds = out.argmax(1)
            ok += (preds == y).sum().item()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(model, loss_log):
    model.train()
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [3]:
augmentation = transforms.RandomAffine(10, scale=(0.9, 1.1), translate=(0.2, 0.2))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
rescale = transforms.Resize((224, 224))

transform = transforms.Compose([
    transforms.ToTensor(),
#     rescale,
    normalize,
])

train_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", 
                            download=True, train=True, transform=transforms.Compose([augmentation, transform]))
valid_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=True, transform=transform)
test_ds = datasets.CIFAR10("/tmp/i291318/CIFAR", download=True, train=False, transform=transform)

train_indices, valid_indices = train_test_split(np.arange(len(train_ds)), test_size=0.2)
train_ds = data.Subset(train_ds, train_indices)
valid_ds = data.Subset(valid_ds, valid_indices)

train_loader = data.DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=10)
valid_loader = data.DataLoader(valid_ds, batch_size=128, shuffle=False, num_workers=10)
test_loader = data.DataLoader(test_ds, batch_size=128, shuffle=False, num_workers=10)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(512, 10),
    nn.LogSoftmax(-1)
)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
validation(model, valid_loader)

(0.1221, 2.3727963035583497)

In [6]:
log_file = open("resnet_log_augmentation_comparison.txt", "w")

## Momentum

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-1, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8)
loss_fn = nn.NLLLoss()

In [8]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(model, loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.2441



Validation accuracy: 0.3182, validation loss: 1.8771
Epoch 2



Training loss: 1.7254



Validation accuracy: 0.4030, validation loss: 1.6584
Epoch 3


...


Validation accuracy: 0.6957, validation loss: 0.8916
Epoch 19



Training loss: 0.7752



Validation accuracy: 0.7276, validation loss: 0.8275
Epoch 20



Training loss: 0.7557



Validation accuracy: 0.7503, validation loss: 0.7255


In [9]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.74865, 0.707124824810028)
Valid: (0.7503, 0.7255343341827393)


In [10]:
optimizer.param_groups[0]["method"] = "RNA"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [11]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.708175, 0.8558816024780274)
Valid: (0.6984, 0.9359578533172608)


In [12]:
optimizer.param_groups[0]["method"] = "RRE"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [13]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.707675, 0.8556134498596192)
Valid: (0.6984, 0.9359554286956787)


In [14]:
final_state = deepcopy(model.state_dict())
# final_optim_state = deepcopy(optimizer.state_dict())

epochs = 10
print("Additional training", file=log_file)

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(model_acc, loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model_acc, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
model.load_state_dict(final_state)
# optimizer.load_state_dict(final_optim_state)
model.cuda()

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-2
print("Main training", file=log_file)

In [ ]:
epochs = 25

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.0986



Validation accuracy: 0.8958, validation loss: 0.3865
Epoch 2



Training loss: 0.0752



Validation accuracy: 0.8988, validation loss: 0.3857
Epoch 3


...


Validation accuracy: 0.9010, validation loss: 0.3963
Epoch 6



Training loss: 0.0516



Validation accuracy: 0.9017, validation loss: 0.4051
Epoch 7



Training loss: 0.0483



Validation accuracy: 0.9009, validation loss: 0.4083
Epoch 8


In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
epochs = 25

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

In [ ]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
print(optimizer.param_groups[0]["method"], flush=True, file=log_file)
model.cpu()
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.cuda()
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
model_acc.cpu()
model.cuda()

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)